# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 26 2022 3:13PM,247311,10,0086006429,ISDIN Corporation,Released
1,Sep 26 2022 3:13PM,247311,10,0086006440,ISDIN Corporation,Released
2,Sep 26 2022 3:13PM,247311,10,0086006454,ISDIN Corporation,Released
3,Sep 26 2022 3:13PM,247311,10,0086006453,ISDIN Corporation,Released
4,Sep 26 2022 3:13PM,247311,10,0086006472,ISDIN Corporation,Released
5,Sep 26 2022 3:13PM,247311,10,0086006505,ISDIN Corporation,Released
6,Sep 26 2022 3:13PM,247311,10,0086006506,ISDIN Corporation,Released
7,Sep 26 2022 3:13PM,247311,10,0086006536,ISDIN Corporation,Released
8,Sep 26 2022 3:13PM,247311,10,0086006555,ISDIN Corporation,Released
9,Sep 26 2022 3:13PM,247311,10,0086006677,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,247303,Released,17
27,247305,Released,3
28,247306,Released,24
29,247308,Released,17
30,247311,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247303,NaN,NaN,17.0
247305,NaN,NaN,3.0
247306,NaN,NaN,24.0
247308,NaN,NaN,17.0
247311,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247223,0.0,0.0,8.0
247238,0.0,0.0,1.0
247240,0.0,1.0,0.0
247254,0.0,0.0,7.0
247258,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247223,0,0,8
247238,0,0,1
247240,0,1,0
247254,0,0,7
247258,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247223,0,0,8
1,247238,0,0,1
2,247240,0,1,0
3,247254,0,0,7
4,247258,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247223,,,8
1,247238,,,1
2,247240,,1,
3,247254,,,7
4,247258,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation
11,Sep 26 2022 2:40PM,247308,19,"AdvaGen Pharma, Ltd"
28,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation
52,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation
55,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC"
72,Sep 26 2022 2:21PM,247302,10,Eywa Pharma Inc.
73,Sep 26 2022 2:15PM,247299,10,Emerginnova
74,Sep 26 2022 2:13PM,247298,22,"NBTY Global, Inc."
75,Sep 26 2022 1:57PM,247295,19,"AdvaGen Pharma, Ltd"
99,Sep 26 2022 1:51PM,247294,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,,,11
1,Sep 26 2022 2:40PM,247308,19,"AdvaGen Pharma, Ltd",,,17
2,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,,,24
3,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,,,3
4,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC",,,17
5,Sep 26 2022 2:21PM,247302,10,Eywa Pharma Inc.,,,1
6,Sep 26 2022 2:15PM,247299,10,Emerginnova,,,1
7,Sep 26 2022 2:13PM,247298,22,"NBTY Global, Inc.",,,1
8,Sep 26 2022 1:57PM,247295,19,"AdvaGen Pharma, Ltd",,17,7
9,Sep 26 2022 1:51PM,247294,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,11,,
1,Sep 26 2022 2:40PM,247308,19,"AdvaGen Pharma, Ltd",17,,
2,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,24,,
3,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,3,,
4,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC",17,,
5,Sep 26 2022 2:21PM,247302,10,Eywa Pharma Inc.,1,,
6,Sep 26 2022 2:15PM,247299,10,Emerginnova,1,,
7,Sep 26 2022 2:13PM,247298,22,"NBTY Global, Inc.",1,,
8,Sep 26 2022 1:57PM,247295,19,"AdvaGen Pharma, Ltd",7,17,
9,Sep 26 2022 1:51PM,247294,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,11,,
1,Sep 26 2022 2:40PM,247308,19,"AdvaGen Pharma, Ltd",17,,
2,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,24,,
3,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,3,,
4,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC",17,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,11.0,NaN,NaN
1,Sep 26 2022 2:40PM,247308,19,"AdvaGen Pharma, Ltd",17.0,NaN,NaN
2,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,24.0,NaN,NaN
3,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,3.0,NaN,NaN
4,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC",17.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,11.0,0.0,0.0
1,Sep 26 2022 2:40PM,247308,19,"AdvaGen Pharma, Ltd",17.0,0.0,0.0
2,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,24.0,0.0,0.0
3,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,3.0,0.0,0.0
4,Sep 26 2022 2:22PM,247303,10,"Snap Medical Industries, LLC",17.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4203728,85.0,1.0,1.0
16,741812,5.0,0.0,0.0
19,989142,26.0,17.0,0.0
20,494555,27.0,14.0,11.0
22,247298,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4203728,85.0,1.0,1.0
1,16,741812,5.0,0.0,0.0
2,19,989142,26.0,17.0,0.0
3,20,494555,27.0,14.0,11.0
4,22,247298,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,85.0,1.0,1.0
1,16,5.0,0.0,0.0
2,19,26.0,17.0,0.0
3,20,27.0,14.0,11.0
4,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,85.0
1,16,Released,5.0
2,19,Released,26.0
3,20,Released,27.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,22
Status,,,,,
Completed,1.0,0.0,0.0,11.0,0.0
Executing,1.0,0.0,17.0,14.0,0.0
Released,85.0,5.0,26.0,27.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,22
0,Completed,1.0,0.0,0.0,11.0,0.0
1,Executing,1.0,0.0,17.0,14.0,0.0
2,Released,85.0,5.0,26.0,27.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,22
0,Completed,1.0,0.0,0.0,11.0,0.0
1,Executing,1.0,0.0,17.0,14.0,0.0
2,Released,85.0,5.0,26.0,27.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()